导入相应的包

In [1]:
import numpy as np
import os
import cv2

查看当前的工作路径

In [2]:
os.getcwd()

'C:\\Users\\marvin\\workspace\\ImageRecongize'

改变当前路径到工作空间

In [3]:
os.chdir('D:\workspace\python\PicRecongize')

读取视频,传入一个视频或者多个图片对象或者一个打开的录像设备ID，返回一个VideoCapture对象

In [4]:
capture = cv2.VideoCapture('m.avi')
print capture

<VideoCapture 0308A560>


获取视频第一帧,返回两个对象，一个是返回一个布尔值（True/False）。如果帧读取的是正确的，就是True。所以最后可以通过检查他的返回值来查看视频文件是否已经到了结尾。第二个是获取当前帧。

In [8]:
ret,first_frame = capture.read()
print ret,first_frame

True [[[ 50  48  44]
  [ 52  50  46]
  [ 53  51  47]
  ..., 
  [154 154 138]
  [ 86  85  74]
  [ 49  48  37]]

 [[110 108 104]
  [111 109 105]
  [111 109 105]
  ..., 
  [148 148 132]
  [ 86  85  74]
  [ 54  53  42]]

 [[174 172 168]
  [174 172 168]
  [174 172 168]
  ..., 
  [144 145 127]
  [ 91  90  77]
  [ 62  61  48]]

 ..., 
 [[ 76  75  77]
  [ 76  75  77]
  [ 76  75  77]
  ..., 
  [143 143 143]
  [116 116 116]
  [115 115 115]]

 [[ 76  75  77]
  [ 76  75  77]
  [ 76  75  77]
  ..., 
  [149 149 149]
  [122 122 122]
  [118 118 118]]

 [[ 76  75  77]
  [ 76  75  77]
  [ 76  75  77]
  ..., 
  [153 153 153]
  [126 126 126]
  [122 122 122]]]


转换颜色空间,传入图片和要转换的颜色类型，返回一个颜色转换后的对象

In [6]:
first_frame_gray = cv2.cvtColor(first_frame,cv2.COLOR_BGR2GRAY)

初始化一个变量，用来统计视频中出现的人数

In [7]:
num=0

In [10]:
while True:
    ret,frame = capture.read()#读取视频的下一帧
    if ret==True:
        
        frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)#将当前读取的帧进行灰度处理

        #计算两个数组或者标量之间每个元素的绝对不同src = |src1 - src2|
        src = cv2.absdiff(frame_gray,first_frame_gray)

        #简单阈值,第一个参数就是原图像，原图像应该是灰度图。第二个参数就是用来对像素值进行分类的阈值。第三个参数
        #就是当像素值高于（有时是小于）阈值时应该被赋予的新的像素值。OpenCV提供了多种不同的阈值方法，这是有第四个
        #参数来决定的。这些方法包括：cv2.THRESH_BINARY(二值阈值化)、cv2.THRESH_BINARY_INV(反向二值阈值化并反转)
        #cv2.THRESH_TRUNC(截断阈值化)、cv2.THRESH_TOZERO（超过阈值被置0）、cv2.THRESH_TOZERO_INV（低于阈值被置0）。
        #函数返回第一个对象是一个最优的阈值，。第二个就是阈值化之后的结果图像。
        retval,threshold_src = cv2.threshold(src,30,255,cv2.THRESH_BINARY)

        #高斯模糊（高斯滤波可以有效的从图像中去除高斯噪音），需要传入第一个参数一张任意通道的图片，但是每个通道
        #需要独立处理，图片有深度限制；第二个参数高斯核大大小，第三个参数是边界类型。返回高斯模糊后的图片
        gass_src = cv2.GaussianBlur(threshold_src,(5,5),1.5)
        #平均（用卷积框覆盖区域所有像素的平均值来代替中心元素），传入第一个参数张任意通道的图片，但是每个通道
        #需要独立处理，图片有深度限制；第二个参数高斯核大大小，第三个参数是边界类型。返回平均化之后的图片
        #gass_src = cv2.blur(threshold_src,(5,5))
        #中值模糊(用与卷积框对应像素的中值来替代中心像素的值),传入第一个参数是一个1或3或5通道的图片，第二个参数
        #光圈的线性大小，值为大于1的奇数。
        #gass_src = cv2.medianBlur(threshold_src,1)

        #定义一个3X3的矩阵
        kernel = np.ones((3,3),np.uint8)
        #膨胀（与卷积核对应的原图像的像素值中只要有一个是1，中心元素的像素值就是1。所以这个操作会增加图像中的白色区域（前景））
        #第一个参数传入一张任意通道的图片，第二个参数传入用于膨胀的元素，如果是一个矩阵的话，则用于膨胀的元素就会是一个3X3的矩阵
        dilate_src = cv2.dilate(gass_src,kernel)

        #轮廓（可以简单认为成将连续的点（连着边界）连在一起的曲线，具有相同的颜色或者灰度）
        #1、为了更加准确，要使用二值化图像。在寻找轮廓之前，要进行阈值化处理或者Canny 边界检测。
        #2、查找轮廓的函数会修改原始图像。如果在找到轮廓之后还想使用原始图像的话，应该将原始图像存储到其他变量中。
        #3、在OpenCV 中，查找轮廓就像在黑色背景中超白色物体。应该记住，要找的物体应该是白色而背景应该是黑色。
        #有三个参数，第一个是输入图像，第二个是轮廓检索模式，第三个是轮廓近似方法。返回值有两个，第一个
        #是轮廓，第二个是（轮廓的）层析结构。轮廓（第一个返回值）是一个Python列表，其中存储这图像中的所有轮廓。每一个
        #轮廓都是一个Numpy 数组，包含对象边界点（x，y）的坐标。
        #对于第三个参数，如果设置成cv2.CHAIN_APPROX_NONE，所有的边界点都会被存储，如果设置成cv2.CHAIN_APPROX_SIMPLE
        #将轮廓上的冗余点都去掉，压缩轮廓，从而节省内存开支。
        contours,hierarchy = cv2.findContours(threshold_src,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        #绘制轮廓（可以根据提供的边界点绘制任何形状），第一个参数是原始图像，第二个参数是轮廓，一个Python 列表。
        #第三个参数是轮廓的索引（在绘制独立轮廓是很有用，当设置为-1 时绘制所有轮廓）。接下来的参数是轮廓的颜色和厚度等
        cv2.drawContours(frame,contours,-1,(0,0,255),2)

        #统计所画轮廓的数量，首先是遍历轮廓，当轮廓面积大于3000的时候画一个矩形并将数量++
        for i in contours:
            x,y,w,h = cv2.boundingRect(i)#boundingRect函数是获得轮廓的坐标和长宽，传入一个轮廓，返回轮廓的坐标和长宽。
            if w*h > 2500 :#判断条件，当轮廓面积大于3000时
                #画一个矩形，传入第一个参数原始图像，第二个参数矩形左上角的坐标，第三个参数矩形右下角的坐标，第四个参数是：形状的颜色
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255))
                num = num+1#统计当前帧的人的数量

        #设置将要在图像上显示的文字变量
        font = "Current number:"
        #将文字放到指定图像位置上，传入第一个参数原始图像，第二个参数是要显示的文字，第三个参数文字显示的位置，第四个参数是设置文字的字体
        #第五个参数是设置字体的颜色
        cv2.putText(frame,font+str(num),(100,50),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255))

        #输出每一帧的人的数量
        print "人数统计:",num

        #当前帧结束后将num重新设置成0
        num =0

        #在窗口上展示图片，窗口会自动调整为图像大小，第一个参数是窗口的名字，第二个是要显示的图像
        cv2.imshow("frame",frame)
        cv2.imshow("first_frame",first_frame)
        #是一个键盘绑定函数。需要指出的是它的时间尺度是毫秒级。函数等待特定的几毫秒，看是否有键盘输入。特定的几毫秒之内，如果
        #按下任意键，这个函数会返回按键的ASCII 码值，程序将会继续运行。如果没有键盘输入，返回值为-1，如果我们设置这个函数的参
        #数为0，那它将会无限期的等待键盘输入。它也可以被用来检测特定键是否被按下，
        if cv2.waitKey(40) & 0xFF == ord('q'):
            break
    else:
        break
#释放capture
capture.release()
#关闭所有窗口
cv2.destroyAllWindows()
        
    
    
    